## Set up

In [1]:
import gc
import pickle
from dataclasses import dataclass
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
import keras as tfk
import keras_tuner
import matplotlib.pyplot as plt
from keras import layers as tfkl
from sklearn.model_selection import train_test_split

2024-12-14 07:09:48.846761: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-14 07:09:48.847764: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-14 07:09:48.887318: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-14 07:09:49.035720: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-14 07:09:49.934593: W tensorflow/compiler/tf2tensorrt/utils/py_uti

In [2]:
IGNORE_BIG_ROCK = False
ONLY_BIG_ROCK = False
DO_HYPERPARAMETER_TUNING = False
OBJECTIVE = "val_big_rock_iou" if ONLY_BIG_ROCK else "val_mean_iou"

In [3]:
SEED = 42
RNG = np.random.default_rng(SEED)
tfk.utils.set_random_seed(SEED)

In [4]:
@dataclass
class Hyperparameter:
    # Data
    big_rock_augmentation_factor: float = 4.0
    augmentation_factor: float = 0.0
    # Structure
    activation: str = "swish"
    filter_size: int = 3
    transpose_filter_size: int = 2
    transpose_stride: int = 2
    max_pool_size: int = 2
    encoder_filters: tuple[int] = (64, 128, 256)
    latent_filters: int = 512
    decoder_filters: tuple[int] = (256, 128, 64)
    learnable_skip_connection: bool = True
    ## Regularisation
    noise: float = 0.0
    downsample_l1: float = 0.0
    downsample_l2: float = 0.0
    upsample_l1: float = 0.0
    upsample_l2: float = 1e-3
    # Training
    optimizer: tfk.Optimizer = tfk.optimizers.Adam
    learning_rate: float = 1e-4
    epochs: int = 500
    validation_split: float = 0.2
    ## Loss
    use_class_weights: bool = True
    ignore_background: bool = True
    dice_weight: float = 0.0
    focal_weight: float = 0.0
    categorical_crossentropy_weight: float = 1.0
    focal_gamma: float = 2.0
    boundary_weight: float = 0.0
    ## Callbacks
    es: bool = True
    es_patience: int = 30
    es_min_delta: float = 1e-4
    lr_factor: float = 0.1
    lr_patience: int = 5
    lr_min_lr: float = 1e-6


hp = Hyperparameter()

## Data

In [5]:
DATA_ROOT = Path("/kaggle/input/an2dl-hw2-clean")
if not DATA_ROOT.exists():
    DATA_ROOT = Path().absolute().parent / "data" / "clean"

DATA_ROOT

PosixPath('/home/tomaz/git/Politecnico/Subjects/deep-learning/an2dl/homework-2/data/clean')

In [6]:
with np.load(DATA_ROOT / "train.npz") as data:
    original_x_train = data["x"]
    original_y_train = data["y"]
with np.load(DATA_ROOT / "test.npz") as data:
    original_x_test = data["x"]

original_x_train = original_x_train[..., np.newaxis] / 255.0
original_y_train = original_y_train[..., np.newaxis]
original_x_test = original_x_test[..., np.newaxis] / 255.0

X_train = original_x_train
y_train = original_y_train
X_test = original_x_test

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Test X shape: {X_test.shape}")

Training X shape: (2505, 64, 128, 1)
Training y shape: (2505, 64, 128, 1)
Test X shape: (10022, 64, 128, 1)


In [7]:
def class_mask(y_train: np.ndarray, class_: int) -> np.ndarray:
    return (y_train == class_).any(axis=(1, 2)).ravel()

In [8]:
if IGNORE_BIG_ROCK and ONLY_BIG_ROCK:
    raise ValueError("Cannot ignore big rock and only big rock at the same time")

if IGNORE_BIG_ROCK:
    print("Ignoring big rock")
    y_train = np.where(y_train == 4, 0, y_train)

big_rock_mask = class_mask(y_train, 4)
if ONLY_BIG_ROCK:
    print("Only big rock")
    y_train = np.where(y_train == 4, 1, np.where(y_train != 4, 0, y_train))
    print(f"Number of images: {big_rock_mask.sum()}")
    X_train = X_train[big_rock_mask]
    y_train = y_train[big_rock_mask]

Ignoring big rock


In [9]:
input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")

Input shape: (64, 128, 1)
Number of classes: 4


In [10]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=hp.validation_split, random_state=SEED
)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(2004, 64, 128, 1) (2004, 64, 128, 1)
(501, 64, 128, 1) (501, 64, 128, 1)


### Augmentation

In [11]:
from scipy.ndimage import rotate


# Define augmentation functions with mask handling
def invert_image(image, mask):
    return 255 - image, mask  # Mask remains unchanged


def add_salt_and_pepper_noise(image, mask, prob=0.03):
    """
    Add salt-and-pepper noise to an image while maintaining its original value range.

    Args:
        image (np.ndarray): Input image.
        mask (np.ndarray): Corresponding mask (unchanged).
        prob (float): Probability of noise per pixel.

    Returns:
        noisy_image (np.ndarray): Image with noise applied.
        mask (np.ndarray): Unchanged mask.
    """
    # Save the original range
    original_min = image.min()
    original_max = image.max()

    # Normalize the image to [0, 1]
    normalized_image = (image - original_min) / (original_max - original_min)

    # Generate random noise map
    random_map = RNG.random(image.shape)
    noisy_image = normalized_image.copy()
    noisy_image[random_map < prob] = 0  # Pepper
    noisy_image[random_map > (1 - prob)] = 1  # Salt

    # Rescale back to the original range
    noisy_image = noisy_image * (original_max - original_min) + original_min
    noisy_image = noisy_image.astype(
        image.dtype
    )  # Ensure the original data type is preserved

    return noisy_image, mask


# Define the rotation function using scipy
def rotate_image_and_mask(image, mask, angle):
    """
    Rotate a grayscale image and its corresponding mask by a given angle.
    """
    # Rotate the grayscale image (bilinear interpolation)
    rotated_image = rotate(image, angle, reshape=False, mode="constant", order=1)

    # Rotate the mask (nearest-neighbor interpolation)
    rotated_mask = rotate(mask, angle, reshape=False, mode="constant", order=0)

    return rotated_image, rotated_mask


def augment_image_and_mask(image, mask, augment_type):
    if augment_type == "Horizontal Flip":
        return np.fliplr(image), np.fliplr(mask)
    elif augment_type == "Vertical Flip":
        return np.flipud(image), np.flipud(mask)
    elif augment_type == "Rotation 20°":
        angle = RNG.uniform(-20, 20)
        rotated_image = rotate(image, angle, reshape=False, mode="constant", order=1)
        rotated_mask = rotate(mask, angle, reshape=False, mode="constant", order=0)
        return rotated_image, rotated_mask
    elif augment_type == "Invert":
        inverted_image, inverted_mask = invert_image(image, mask)
        return inverted_image, inverted_mask
    elif augment_type == "noise":
        noisy_image, noisy_mask = add_salt_and_pepper_noise(image, mask)
        return noisy_image, noisy_mask

    return image, mask


def augment(
    images: np.ndarray, masks: np.ndarray, augmentations, factor: float = 1.0
) -> tuple[np.ndarray, np.ndarray]:
    if images.shape[0] != masks.shape[0]:
        raise ValueError()
    samples = images.shape[0]
    augmented_images, augmented_masks = [], []
    for i in RNG.choice(samples, size=int(samples * factor), replace=True):
        augmented_image, augmented_mask = augment_image_and_mask(
            images[i],
            masks[i],
            RNG.choice(augmentations),
        )
        augmented_images.append(augmented_image)
        augmented_masks.append(augmented_mask)

    return np.array(augmented_images), np.array(augmented_masks)


In [12]:
# https://keras.io/examples/vision/mixup/
def mixup(image, mask, new_image, new_mask, alpha=0.2):
    mixup_factor = RNG.beta(alpha, alpha, size=image.shape)
    image = image * mixup_factor + new_image * (1 - mixup_factor)
    mask = mask * mixup_factor + new_mask * (1 - mixup_factor)
    return image, mask

In [13]:
AUGMENTATIONS = (
    "Horizontal Flip",
    "Vertical Flip",
    "Rotation 20°",
    "noise",
)

### Augment big rock class

In [14]:
big_rock_value = 1 if ONLY_BIG_ROCK else 4
new_big_rock_mask = class_mask(y_train, big_rock_value)
new_big_rock_mask.shape

(2004,)

In [15]:
X_augmented, y_augmented = augment(
    X_train,
    y_train,
    # X_train[new_big_rock_mask],
    # y_train[new_big_rock_mask],
    AUGMENTATIONS,
    factor=hp.big_rock_augmentation_factor,
)

print(X_augmented.shape, y_augmented.shape)

(8016, 64, 128, 1) (8016, 64, 128, 1)


#### Mix-up augmentation

In [16]:
# big_rock_indices = np.nonzero(new_big_rock_mask)[0]
# non_big_rock_indices = np.nonzero(~big_rock_mask)[0]
# X_mixup, y_mixup = [], []
# for i in RNG.choice(non_big_rock_indices, size=50, replace=False):
#     big_rock_index = RNG.choice(big_rock_indices)
#     x, y = mixup(
#         X_train[big_rock_index], y_train[big_rock_index],
#         original_x_train[i], np.zeros_like(original_y_train[i]),
#         alpha=0.05,
#     )
#     X_mixup.append(x)
#     y_mixup.append(y)

# X_mixup, y_mixup = np.array(X_mixup), np.array(y_mixup)

# print(X_mixup.shape, y_mixup.shape)

In [17]:
# X_augmented = np.concatenate((X_augmented, X_mixup), axis=0)
# y_augmented = np.concatenate((y_augmented, y_mixup), axis=0)

# print(X_augmented.shape, y_augmented.shape)

### Merge augmented datasets

In [18]:
X_train = np.concatenate((X_train, X_augmented), axis=0)
y_train = np.concatenate((y_train, y_augmented), axis=0)

print(X_train.shape, y_train.shape)

(10020, 64, 128, 1) (10020, 64, 128, 1)


### Class weights

In [19]:
# Convert y_train to integers for class count
y_train_int = y_train.astype(np.int32)

# Calculate class weights based on pixel proportions
class_pixel_counts = np.bincount(
    y_train_int.flatten(), minlength=num_classes
)  # Count pixels for each class
total_pixels = np.sum(class_pixel_counts)  # Total number of pixels
class_weights = total_pixels / (class_pixel_counts + 1e-6)  # Inverse frequency
class_weights /= np.sum(class_weights)  # Normalize to sum to 1

print(f"Class pixel counts: {class_pixel_counts}")
print(f"Calculated class weights: {class_weights}")

Class pixel counts: [20917731 27118292 19134650 14913167]
Calculated class weights: [0.23434722 0.18076405 0.2561851  0.32870363]


## Model

In [20]:
class VisualizeSegmentationCallback(tfk.callbacks.Callback):
    def __init__(self, X_train, y_train, min_number_classes=4, num_images=2):
        super().__init__()
        self.X_train = X_train
        self.y_train = y_train
        self.num_images = num_images
        self.selected_indices = [
            i
            for _, (i, y) in zip(range(num_images), enumerate(y_train))
            if np.unique(y).size >= min_number_classes
        ]

    def on_epoch_end(self, epoch, logs=None):
        # Plot predictions for the selected images
        _, axes = plt.subplots(self.num_images, 3, figsize=(15, self.num_images * 5))

        for idx, i in enumerate(self.selected_indices):
            # Extract image and ground truth
            X_sample = self.X_train[i : i + 1]  # Add batch dimension
            y_sample = self.y_train[i]

            # Predict on the image
            predicted_mask = self.model.predict(X_sample, verbose=0)
            predicted_mask = np.argmax(predicted_mask, axis=-1)[0]

            # Visualize the input, ground truth, and predicted mask
            axes[idx, 0].imshow(X_sample[0].squeeze(), cmap="gray")
            axes[idx, 0].set_title("Input Image")
            axes[idx, 0].axis("off")

            axes[idx, 1].imshow(y_sample, cmap="viridis", vmin=0, vmax=4)
            axes[idx, 1].set_title("Ground Truth Mask")
            axes[idx, 1].axis("off")

            axes[idx, 2].imshow(predicted_mask, cmap="viridis", vmin=0, vmax=4)
            axes[idx, 2].set_title(f"Predicted Mask (Epoch {epoch + 1})")
            axes[idx, 2].axis("off")

        plt.tight_layout()
        plt.show()

### Loss function

#### Dice loss

In [48]:
def dice_loss(
    y_true: tf.Tensor,
    y_pred: tf.Tensor,
    weights: list[float],
    ignore_background: bool,
) -> tf.Tensor:
    dice_loss_per_image = tf.constant(0.0, shape=y_true.shape[0])
    for class_, weight in enumerate(weights):
        if ignore_background and class_ == 0:
            continue
        y_true_class = y_true[:, :, :, class_]
        y_pred_class = y_pred[:, :, :, class_]
        numerator = 2 * tf.reduce_sum(y_true_class * y_pred_class, axis=(1, 2))
        denominator = tf.reduce_sum(y_true_class + y_pred_class, axis=(1, 2))
        dice_loss_per_image += weight * (
            1 - numerator / (denominator + tfk.backend.epsilon())
        )
    # Return the mean Dice loss across the batch
    return tf.cast(tf.reduce_mean(dice_loss_per_image), tf.float32)

#### Boundary loss

In [49]:
# # Simple script which includes functions for calculating surface loss in keras
# ## See the related discussion: https://github.com/LIVIAETS/boundary-loss/issues/14

from scipy.ndimage import distance_transform_edt as distance


def calc_dist_map(seg):
    res = np.zeros_like(seg)
    posmask = seg.astype(bool)

    if posmask.any():
        negmask = ~posmask
        res = distance(negmask) * negmask - (distance(posmask) - 1) * posmask

    return res


def calc_dist_map_batch(y_true) -> np.ndarray:
    y_true_numpy = y_true.numpy()
    return (
        np.array([calc_dist_map(y) for y in y_true_numpy])
        .reshape(y_true.shape)
        .astype(np.float32)
    )


def boundary_loss(y_true, y_pred):
    y_true_dist_map = tf.py_function(
        func=calc_dist_map_batch, inp=[y_true], Tout=tf.float32
    )
    multiplied = y_pred * y_true_dist_map
    return tf.reduce_mean(multiplied)


# # Scheduler
# ### The following scheduler was proposed by @marcinkaczor
# ### https://github.com/LIVIAETS/boundary-loss/issues/14#issuecomment-547048076

# class AlphaScheduler(Callback):
#     def __init__(self, alpha, update_fn):
#         self.alpha = alpha
#         self.update_fn = update_fn
#     def on_epoch_end(self, epoch, logs=None):
#         updated_alpha = self.update_fn(K.get_value(self.alpha))
#         K.set_value(self.alpha, updated_alpha)


# alpha = K.variable(1, dtype='float32')

# def gl_sl_wrapper(alpha):
#     def gl_sl(y_true, y_pred):
#         return alpha * generalized_dice_loss(
#             y_true, y_pred) + (1 - alpha) * surface_loss_keras(y_true, y_pred)
#     return gl_sl

# model.compile(loss=gl_sl_wrapper(alpha))

# def update_alpha(value):
#   return np.clip(value - 0.01, 0.01, 1)

# history = model.fit_generator(
#   ...,
#   callbacks=AlphaScheduler(alpha, update_alpha)
# )

#### Hybrid loss

In [59]:
# def combined_loss(
#     weights: list[float],
#     dice_weight: float,
#     focal_weight: float,
#     categorical_crossentropy_weight: float,
#     boundary_weight: float,
#     alpha: float | list[float],
#     gamma: float,
#     ignore_background: bool = False,
# ):
#     def loss(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
#         y_true = tf.squeeze(y_true, axis=-1)

#         mask = tf.ones_like(y_true, dtype=tf.float32)
#         if ignore_background:
#             mask = tf.cast(y_true != 0, tf.float32)

#         # One-hot encode class labels since they are dense
#         # but predictions are sparse
#         y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=len(weights))

#         y_true_masked = y_true * mask[..., tf.newaxis]
#         y_pred_masked = y_pred * mask[..., tf.newaxis]

#         categorical_crossentropy_value = tfk.losses.categorical_crossentropy(
#             y_true_masked,
#             y_pred_masked,
#         )
#         dice_loss_value = dice_loss(y_true, y_pred, weights, ignore_background)
#         focal_loss_value = tf.cast(
#             tfk.losses.categorical_focal_crossentropy(
#                 y_true_masked,
#                 y_pred_masked,
#                 alpha=alpha,
#                 gamma=gamma,
#             ),
#             tf.float32,
#         )

#         # FIXME: Add background mask
#         boundary_loss_value = 0.0
#         if boundary_weight > 0.0:
#             boundary_loss_value = boundary_loss(y_true, y_pred)

#         return (
#             dice_weight * dice_loss_value
#             + focal_weight * focal_loss_value
#             + categorical_crossentropy_weight * categorical_crossentropy_value
#             + boundary_weight * boundary_loss_value
#         )

#     return loss

In [78]:
def combined_loss(
    weights: list[float],
    dice_weight: float,
    focal_weight: float,
    categorical_crossentropy_weight: float,
    boundary_weight: float,
    alpha: float | list[float],
    gamma: float,
    ignore_background: bool = False,
):
    def loss(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
        y_true = tf.squeeze(y_true, axis=-1)

        mask = tf.ones_like(y_true, dtype=tf.float32)
        if ignore_background:
            mask = tf.cast(y_true != 0, tf.float32)
        mask_sum = tf.reduce_sum(mask) + tf.keras.backend.epsilon()

        # One-hot encode class labels since they are dense but predictions are sparse
        y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), depth=len(weights))

        categorical_crossentropy = (
            -y_true_one_hot
            * tf.math.log(y_pred + tf.keras.backend.epsilon())
            * weights
        )

        # We already weigh the categorical cross-entropy,
        # so we don't need alpha
        focal_loss_value = tf.reduce_sum(
            (1 - y_pred) ** gamma * categorical_crossentropy, axis=-1
        )
        categorical_crossentropy_value = tf.reduce_sum(
            categorical_crossentropy, axis=-1
        )

        # Apply the mask to (possibly) exclude background contributions
        categorical_crossentropy_value *= mask
        focal_loss_value *= mask

        # Aggregate the losses, normalizing by the sum of the mask
        pixel_loss_value = (
            focal_weight * tf.reduce_sum(focal_loss_value)
            + categorical_crossentropy_weight
            * tf.reduce_sum(categorical_crossentropy_value)
        ) / mask_sum

        dice_loss_value = dice_loss(y_true_one_hot, y_pred, weights, ignore_background)

        total_loss = pixel_loss_value + dice_weight * dice_loss_value

        return total_loss

    return loss

### Architecture

In [79]:
@tfk.utils.register_keras_serializable(package="Custom")
def residual_block(x, filters, size, activation, l1=0.0, l2=0.0):
    shortcut = x
    for _ in range(2):
        x = tfkl.Conv2D(filters, size, padding="same", kernel_regularizer=tfk.regularizers.L1L2(l1=l1, l2=l2))(x)
        x = tfkl.LayerNormalization()(x)
        x = tfkl.Activation(activation)(x)
    shortcut = tfkl.Conv2D(filters, (1, 1), padding="same")(shortcut)
    return tfkl.add([x, shortcut])

@tfk.utils.register_keras_serializable(package="Custom")
def learnable_skip_connection(encoder_features, decoder_features, filters):
    concat_features = tfkl.Concatenate()([encoder_features, decoder_features])
    fusion_gate = tfkl.Conv2D(filters, (1, 1), activation="sigmoid")(concat_features)
    weighted_features = fusion_gate * encoder_features + (1 - fusion_gate) * decoder_features
    return weighted_features


# U-Net with Residual Connections
def enhanced_unet(hp: Hyperparameter, input_shape, num_classes):
    inputs = tfkl.Input(input_shape)
    x = inputs

    x = tfkl.GaussianNoise(hp.noise)(x)

    # Encoder
    encoder_outputs = []
    for encoder_filters in hp.encoder_filters:
        x = residual_block(
            x,
            encoder_filters,
            (hp.filter_size, hp.filter_size),
            hp.activation,
            hp.downsample_l1,
            hp.downsample_l2,
        )
        encoder_outputs.append(x)
        x = tfkl.MaxPooling2D((hp.max_pool_size, hp.max_pool_size))(x)

    x = residual_block(
        x, hp.latent_filters, (hp.filter_size, hp.filter_size), hp.activation
    )

    # Decoder
    for i, decoder_filters in enumerate(hp.decoder_filters):
        x = tfkl.Conv2DTranspose(
            decoder_filters,
            (hp.transpose_filter_size, hp.transpose_filter_size),
            strides=(hp.transpose_stride, hp.transpose_stride),
            padding="same",
        )(x)
        if hp.learnable_skip_connection:
            x = learnable_skip_connection(encoder_outputs[-(i + 1)], x, decoder_filters)
        else:
            x = tfkl.Concatenate()([x, encoder_outputs[-(i + 1)]])
        x = residual_block(
            x,
            decoder_filters,
            (hp.filter_size, hp.filter_size),
            hp.activation,
            hp.upsample_l1,
            hp.upsample_l2,
        )

    outputs = tfkl.Conv2D(num_classes, (1, 1), activation="softmax")(x)

    return tfk.Model(inputs, outputs)

In [80]:
def build(hp: Hyperparameter, class_weights: np.ndarray) -> tuple[tfk.Model, dict]:
    if not hp.use_class_weights:
        print("Using constant class weights")
        class_weights = np.ones_like(class_weights) / num_classes

    if not np.allclose(class_weights.sum(), 1):
        raise ValueError("Class weights must sum to 1")

    input_shape = X_train.shape[1:]
    model = enhanced_unet(hp, input_shape, num_classes)

    loss = combined_loss(
        class_weights,
        dice_weight=hp.dice_weight,
        focal_weight=hp.focal_weight,
        categorical_crossentropy_weight=hp.categorical_crossentropy_weight,
        boundary_weight=hp.boundary_weight,
        alpha=class_weights,
        gamma=hp.focal_gamma,
        ignore_background=hp.ignore_background,
    )
    # Boundary loss makes a call to a python function
    # so JIT compilation must be disabled if it is used
    jit_compile = True
    if hp.boundary_weight > 0:
        jit_compile = False

    metrics = [
            "accuracy",
            tfk.metrics.MeanIoU(
                num_classes=num_classes,
                name="mean_iou",
                ignore_class=0,
                sparse_y_pred=False,
            ),
    ]
    if not IGNORE_BIG_ROCK:
        metrics.append(
            tfk.metrics.IoU(
                num_classes=num_classes,
                target_class_ids=(big_rock_value,),
                name="big_rock_iou",
                sparse_y_pred=False,
            )
        )

    model.compile(
        optimizer=hp.optimizer(learning_rate=hp.learning_rate),
        loss=loss,
        metrics=metrics,
        jit_compile=jit_compile,
    )

    custom_objects = {
        "loss": loss
    }

    return model, custom_objects

In [81]:
def callbacks(hp: Hyperparameter):
    reduce_lr = tfk.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=hp.lr_factor,
        patience=hp.lr_patience,
        min_lr=hp.lr_min_lr,
        verbose=1,
    )

    visualize_callback = VisualizeSegmentationCallback(
        X_train, y_train, min_number_classes=2
    )

    checkpoint = tfk.callbacks.ModelCheckpoint(
        "model.keras",
        monitor=OBJECTIVE,
        save_best_only=True,
        mode="max",
        verbose=1,
    )

    callback_list = [
        reduce_lr,
        visualize_callback,
        checkpoint,
    ]
    if hp.es:
        early_stopping = tfk.callbacks.EarlyStopping(
            monitor="val_loss",
            restore_best_weights=True,
            patience=hp.es_patience,
            min_delta=hp.es_min_delta,
            verbose=1,
        )
        callback_list.append(early_stopping)

    return callback_list

In [82]:
model, custom_objects = build(hp, class_weights)
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 64, 128,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gaussian_noise_2    │ (None, 64, 128,   │          0 │ input_layer_2[0]… │
│ (GaussianNoise)     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_50 (Conv2D)  │ (None, 64, 128,   │        640 │ gaussian_noise_2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 128,   │        128 │ conv2d_50[0][0]   │
│ (LayerNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_28       │ (None, 64, 128,   │          0 │ layer_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_51 (Conv2D)  │ (None, 64, 128,   │     36,928 │ activation_28[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 128,   │        128 │ conv2d_51[0][0]   │
│ (LayerNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_29       │ (None, 64, 128,   │          0 │ layer_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_52 (Conv2D)  │ (None, 64, 128,   │        128 │ gaussian_noise_2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_20 (Add)        │ (None, 64, 128,   │          0 │ activation_29[0]… │
│                     │ 64)               │            │ conv2d_52[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 32, 64,    │          0 │ add_20[0][0]      │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_53 (Conv2D)  │ (None, 32, 64,    │     73,856 │ max_pooling2d_6[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 32, 64,    │        256 │ conv2d_53[0][0]   │
│ (LayerNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_30       │ (None, 32, 64,    │          0 │ layer_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_54 (Conv2D)  │ (None, 32, 64,    │    147,584 │ activation_30[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 32, 64,    │        256 │ conv2d_54[0][0]   │
│ (LayerNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_31       │ (None, 32, 64,    │          0 │ layer_normalizat

 Total params: 7,359,876 (28.08 MB)

 Trainable params: 7,359,876 (28.08 MB)

 Non-trainable params: 0 (0.00 B)

### Hyperparameter tuning

In [83]:
def build_wrapper(tuner_hp: keras_tuner.HyperParameters) -> tfk.Model:
    tfk.backend.clear_session()
    gc.collect()

    hp = Hyperparameter()
    hp.focal_weight = tuner_hp.Float(
        name="focal_weight",
        default=hp.focal_weight,
        min_value=0.0,
        max_value=1.0,
    )
    hp.focal_gamma = tuner_hp.Int(
        name="focal_gamma",
        default=hp.focal_gamma,
        min_value=2,
        max_value=20,
    )
    hp.dice_weight = tuner_hp.Float(
        name="dice_weight",
        default=hp.dice_weight,
        min_value=0.0,
        max_value=1.0,
    )
    hp.boundary_weight = tuner_hp.Float(
        name="boundary_weight",
        default=hp.boundary_weight,
        min_value=0.0,
        max_value=1.0,
    )

    return build(hp, class_weights)[0]

In [84]:
tuner = keras_tuner.Hyperband(
    build_wrapper,
    objective=keras_tuner.Objective(OBJECTIVE, direction="max"),
    max_epochs=hp.epochs,
    factor=3,
    directory="optimisation_results",
    seed=SEED,
    hyperband_iterations=2,
)
tuner.search_space_summary()

Search space summary
Default search space size: 4
focal_weight (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 1.0, 'step': None, 'sampling': 'linear'}
focal_gamma (Int)
{'default': 2.0, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
dice_weight (Float)
{'default': 1.0, 'conditions': [], 'min_value': 0.0, 'max_value': 1.0, 'step': None, 'sampling': 'linear'}
boundary_weight (Float)
{'default': 1.0, 'conditions': [], 'min_value': 0.0, 'max_value': 1.0, 'step': None, 'sampling': 'linear'}


In [85]:
if DO_HYPERPARAMETER_TUNING:
    tuner.search(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        callbacks=callbacks(hp),
        epochs=hp.epochs,
        verbose=2,
    )

    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = build_wrapper(best_hp)
else:
    # BIG ROCK
    # # Optimisation results 1
    # hp.focal_weight = 0.022732
    # hp.focal_gamma = 2
    # hp.dice_weight = 0.92191
    # hp.boundary_weight = 0.47939

    # # Optimisation results 2
    # hp.focal_weight = 0.71098
    # hp.focal_gamma = 19
    # hp.dice_weight = 0.5194
    # hp.boundary_weight = 0.087301

    # COMBINED
    # # Optimisation results 1
    # hp.focal_weight = 0.84689
    # hp.focal_gamma =  11
    # hp.dice_weight = 0.97885
    # # hp.boundary_weight = 0.19206
    # hp.boundary_weight = 0.0

    model, custom_objects = build(hp, class_weights)

## Training

In [86]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    callbacks=callbacks(hp),
    epochs=hp.epochs,
    verbose=2,
)

Epoch 1/500


KeyboardInterrupt: 

## Evaluation

In [ ]:
with Path("history.pkl").open("wb") as f:
    pickle.dump(history.history, f)
print("Saved training history")

In [ ]:
y_hat_val = model.predict(X_val, verbose=0)
y_hat_val = np.argmax(y_hat_val, axis=-1)

np.savez(
    Path("val"),
    x=X_val,
    y=y_val,
    y_hat=y_hat_val,
)

print("Saved validation predictions")

## Submission

In [ ]:
MODEL_PATH = Path("model.keras")
if not MODEL_PATH.exists():
    MODEL_PATH = Path("/kaggle") / "working" / "model.keras"

In [ ]:
model: tfk.Model = tfk.models.load_model(
    MODEL_PATH,
    custom_objects=custom_objects,
)

In [ ]:
# Generate predictions
predicted_masks = model.predict(X_test, verbose=0)
predicted_masks = np.argmax(
    predicted_masks, axis=-1
)  # Convert probabilities to class labels
print(f"Predictions shape: {predicted_masks.shape}")

In [ ]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [ ]:
# Create and download the csv submission file
submission_filename = "submission.csv"
submission_df = y_to_df(predicted_masks)
submission_df.to_csv(submission_filename, index=False)